# HAL-9000 intent classification demo

In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import tensorflow as tf
from nltk.tokenize import word_tokenize
import nltk
from pymagnitude import *
from nltk.stem.lancaster import LancasterStemmer

## load intents dataset

In [2]:
X = []
y = []
with open("/Users/zetong/intents.csv", mode = 'r', encoding = 'ascii', errors = 'ignore') as csvfile:
    intents = pd.read_csv(csvfile)
    X = list(intents["utterances"])
    y = list(intents["labels"])
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)
X = np.asarray(X)

In [3]:
le.classes_.shape

(21,)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [5]:
vectors = Magnitude("/Users/zetong/Downloads/glove.840B.300d.magnitude")

In [6]:
MAX_SEQ_LEN = 28
i = tf.keras.layers.Input(shape=(MAX_SEQ_LEN, vectors.dim))
Bidir_LSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(i)
hidden = tf.keras.layers.Dense(16)(Bidir_LSTM)
dropout = tf.keras.layers.Dropout(0.3)(hidden)
output = tf.keras.layers.Dense(le.classes_.shape[0], activation="softmax")(dropout)
model = tf.keras.Model(inputs=i, outputs=output)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])
model.summary()

W0909 17:58:41.645711 4582110656 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0909 17:58:41.651076 4582110656 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0909 17:58:41.652842 4582110656 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensor

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 300)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                85248     
_________________________________________________________________
dense (Dense)                (None, 16)                1040      
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 21)                357       
Total params: 86,645
Trainable params: 86,645
Non-trainable params: 0
_________________________________________________________________


In [7]:
def make_dataset(xarr, yarr):
    dataset = tf.data.Dataset.from_tensor_slices((xarr, yarr)).repeat()
    
    def _process_string(x):

        # x is numpy array
        def _pad_zeros(x, MAX_SEQ_LEN):
            if x.shape[0] >= MAX_SEQ_LEN:
                return x[0:MAX_SEQ_LEN, :]
            else:
                return np.concatenate(
                    (x, np.zeros((MAX_SEQ_LEN - x.shape[0], x.shape[1]))), axis=0
                )
        stemmer = LancasterStemmer()
        x = x.numpy().decode()
        x = word_tokenize(x)
        x = [stemmer.stem(i) for i in x]
        if len(x) != 0:
            x = vectors.query(x)
            x = _pad_zeros(x, MAX_SEQ_LEN)
        else:
            x = np.zeros((MAX_SEQ_LEN, vectors.dim))
        return x
    
    def _process_datapair(X, y):
        X = tf.py_function(_process_string, [X], tf.float32)
        X.set_shape([MAX_SEQ_LEN, vectors.dim])
        y.set_shape([])
        return X, y
    
    dataset = dataset.map(_process_datapair)
    return dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(batch_size)

In [8]:
batch_size = 32
train = make_dataset(X_train, y_train)
val = make_dataset(X_val, y_val)
test = make_dataset(X_test, y_test)

In [9]:
model.fit(train, validation_data=val, validation_steps = X_val.shape[0] / batch_size, steps_per_epoch=200, epochs=10)

W0909 17:58:45.767891 4582110656 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10


W0909 17:58:47.778912 123145553641472 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0909 17:58:47.796681 123145546665984 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0909 17:58:47.810189 123145546665984 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0909 17:58:47.817901 123145553641472 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0909 17:58:47.824609 123145546665984 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string


200/200 [==============================] - 14s 70ms/step - loss: 2.2232 - acc: 0.3356 - val_loss: 1.6622 - val_acc: 0.4896
Epoch 2/10
200/200 [==============================] - 9s 43ms/step - loss: 0.9328 - acc: 0.7414 - val_loss: 0.8374 - val_acc: 0.7812
Epoch 3/10
200/200 [==============================] - 8s 40ms/step - loss: 0.5064 - acc: 0.8578 - val_loss: 0.9810 - val_acc: 0.7396
Epoch 4/10
200/200 [==============================] - 8s 41ms/step - loss: 0.3294 - acc: 0.9069 - val_loss: 0.7133 - val_acc: 0.8438
Epoch 5/10
200/200 [==============================] - 8s 41ms/step - loss: 0.2324 - acc: 0.9344 - val_loss: 1.0254 - val_acc: 0.8021
Epoch 6/10
200/200 [==============================] - 8s 41ms/step - loss: 0.1862 - acc: 0.9483 - val_loss: 0.9728 - val_acc: 0.7917
Epoch 7/10
200/200 [==============================] - 8s 41ms/step - loss: 0.1557 - acc: 0.9584 - val_loss: 0.8001 - val_acc: 0.8229
Epoch 8/10
200/200 [==============================] - 9s 45ms/step - loss: 0.11

In [10]:
model.evaluate(test, steps=X_val.shape[0] / batch_size)

3/2 [================================] - 2s 691ms/step - loss: 1.0060 - acc: 0.8438


[1.0642372731412395, 0.84375]